## Sentiment Analysis of Reviews using RNNs in TensorFlow, with pre-built embeddings


In [1]:
import collections
import math
import os
import random
import tarfile
import re

In [2]:
from six.moves import urllib

In [3]:
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
import tensorflow as tf

/home/mayank/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
print(np.__version__)
print(mp.__version__)
print(tf.__version__)

1.14.2
2.2.2
1.6.0


#### Download, unzip and untar files in an automated way

In [5]:
DOWNLOADED_FILENAME = 'ImdbReviews.tar.gz'

def download_file(url_path):
    if not os.path.exists(DOWNLOADED_FILENAME):
        filename, _ = urllib.request.urlretrieve(url_path, DOWNLOADED_FILENAME)

    print('Found and verified file from this path: ', url_path)
    print('Downloaded file: ', DOWNLOADED_FILENAME)

### Extract reviews and the corresponding positive and negative labels from the dataset

In [6]:
TOKEN_REGEX = re.compile("[^A-Za-z0-9 ]+")


def get_reviews(dirname, positive=True):
    label = 1 if positive else 0

    reviews = []
    labels = []
    for filename in os.listdir(dirname):
        if filename.endswith(".txt"):
            with open(dirname + filename, 'r+') as f:
                review = f.read()
                review = review.lower().replace("<br />", " ")
                review = re.sub(TOKEN_REGEX, '', review)
                
                reviews.append(review)
                labels.append(label)
    
    return reviews, labels           

def extract_labels_data():
    # If the file has not already been extracted
    if not os.path.exists('aclImdb'):
        with tarfile.open(DOWNLOADED_FILENAME) as tar:
            tar.extractall()
            tar.close()
        
    positive_reviews, positive_labels = get_reviews("aclImdb/train/pos/", positive=True)
    negative_reviews, negative_labels = get_reviews("aclImdb/train/neg/", positive=False)

    data = positive_reviews + negative_reviews
    labels = positive_labels + negative_labels

    return labels, data

In [7]:
URL_PATH = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

download_file(URL_PATH)

Found and verified file from this path:  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Downloaded file:  ImdbReviews.tar.gz


In [8]:
labels, data = extract_labels_data()

In [9]:
labels[:5]

[1, 1, 1, 1, 1]

In [10]:
data[:5]

['excellent episode movie ala pulp fiction 7 days  7 suicides it doesnt get more depressing than this movie rating 810 music rating 1010',
 'ive just read the most recent remarks about this movie and i would like to respond youre probably not familiar with the original story of rap group nwa which dates back to the beginning in 1988 in 1989 ice cube left the band to go solo and ultimately in 1991 the band breaking up when drdre left which led to a lot of beef starting with the departure of ice cube and drdre in 1991 this story was somewhat based on that  further more this movie was a 90 minute laughing spree the way they explained the bootie juice song to be a political statement was hilarious not to mention the love song tasty was hooking up and when vanilla sherbert got his ass kicked just like the record company executive is also hilarious and having theyre managers getting shot every time too  people who didnt enjoy this movie probably didnt get it or were complete idiots my opinio

In [11]:
len(labels), len(data)

(25000, 25000)

In [12]:
max_document_length = max([len(x.split(" ")) for x in data])
print(max_document_length)

2470


### How many words to consider in each review?

Majority of the reviews fall under 250 words. This a number we've chosen based on some analysis of the data:

* Count the number of words in each file and divide by number of files to get an average i.e. **avg_words_per_file = total_words / num_files**
* Plot the words per file on matplot lib and try find a number which includes a majority of files

Word embeddings all have the same dimensionality which you can specify. A document is a vector of word embeddings (one dbpedia instance is a document in this case)

* Each document should be of the **same length**, documents longer than the MAX_SEQUENCE_LENGTH are truncated to this length
* The other documents will be **padded** by a special symbol to be the same max length

In [13]:
MAX_SEQUENCE_LENGTH = 250

### Use a pre-trained model for embeddings

Instead of training our model on our own dataset we will use a pre-trained model.

This is much better because these word vectors will be more generalized as they have been trained on a different dataset. These embeddings are trained using GloVe, a vector generation model very simalar to word2vec. 

In [19]:
words = np.load('wordsList.npy')

words = [w.decode('utf-8') for w in words]

In [20]:
words[:5], len(words)

(['0', ',', '.', 'of', 'to'], 400000)

### Map every word to a unique index

The words are in the order and the position of the word in the word list is its index.

In [21]:
def get_word_index_dictionary(words):
    
    dictionary = {}
    
    index = 0
    for word in words:
        dictionary[word] = index
        index += 1
    
    return dictionary

dictionary = get_word_index_dictionary(words)        

#### The most common words have lower index values

In [22]:
dictionary['and'], dictionary['this'], dictionary['together'], dictionary['supreme']

(5, 37, 600, 1399)

### Convert the sentences so they're represented in the form of word indexes

Use the word index mapping that we created earlier in order to look up the index for individual words

In [25]:
review_ids = []

def convert_reviews_to_ids(data, words):
    words_list = words

    progress = 0
    for review in data:
        review_id = []
        
        index = 0
        for word in review:
            if index >= MAX_SEQUENCE_LENGTH:
                break;
            
            try:
                review_id.append(dictionary[word])
            except KeyError:
                review_id.append(0)
            
            index += 1
        if len(review_id) < MAX_SEQUENCE_LENGTH:
            review_id = np.pad(review_id, (0, MAX_SEQUENCE_LENGTH - index), 'constant')

        review_ids.append(np.array(review_id))
        progress += 1
        
        if progress % 1000 == 0:
            print("Completed: ", progress)

In [26]:
convert_reviews_to_ids(data, words)

Completed:  1000
Completed:  2000
Completed:  3000
Completed:  4000
Completed:  5000
Completed:  6000
Completed:  7000
Completed:  8000
Completed:  9000
Completed:  10000
Completed:  11000
Completed:  12000
Completed:  13000
Completed:  14000
Completed:  15000
Completed:  16000
Completed:  17000
Completed:  18000
Completed:  19000
Completed:  20000
Completed:  21000
Completed:  22000
Completed:  23000
Completed:  24000
Completed:  25000


In [27]:
review_ids[19825]

array([  41,    0, 1911, 1110,    7, 5025, 5025, 3524,    0, 1968,   41,
       1534, 5025,   41, 4652, 1110, 1968,    0, 2159, 5918,   41, 1534,
          0, 1993, 4868, 2404,   41, 1110, 1993,    7,   41, 3814, 5025,
       3524,    0, 1556, 1110, 1864,    7, 6479, 1534, 1110,    0, 4868,
       3880,    0, 2159, 5918, 1110,    0, 1993,    7,   41, 3814,    0,
       1864, 5918,    7, 1911,    7, 1864, 2159, 1110, 1911, 1534,    0,
       2159, 5918, 1110, 3524,    0,    7, 1911, 1110,    0, 1556, 4868,
       2159, 5918,    0,   41, 1993, 1993,    7, 2159, 6479, 1911, 1110,
          0, 1534, 1110, 5025, 3880,   41, 1534, 5918,    0,    7, 3814,
       1968,    0, 1534, 1110, 5025, 3880, 1864, 1110, 3814, 2159, 1110,
       1911, 1110, 1968,    0, 3420, 1110, 4868, 3420, 5025, 1110,    0,
       2159, 5918, 1110, 3524,    0, 5918, 6479, 1911, 2159,    0, 1110,
       2404, 1110, 1911, 3524, 1556, 4868, 1968, 3524,    0,    7, 1911,
       4868, 6479, 3814, 1968,    0, 2159, 5918, 11

### Load this saved file to get the reviews in the IMDB dataset represented using word indexes

These have been pre-calculated and saved, and will help you if your id mapping code takes too long to run

In [28]:
review_ids = np.load('idsMatrix.npy')

In [29]:
review_ids.shape

(25000, 250)

In [30]:
review_ids[:5]

array([[174943,    152,     14, ...,      0,      0,      0],
       [ 26494,     46, 399999, ...,   2153,    144,      7],
       [  6520, 399999,     21, ...,      0,      0,      0],
       [    37,     14,   2407, ...,      0,      0,      0],
       [    37,     14,     36, ...,      0,      0,      0]], dtype=int32)

In [31]:
x_data = review_ids
y_output = np.array(labels)

vocabulary_size = len(words)
print(vocabulary_size)

400000


In [32]:
data[3:5]

['airwolf the movie a variation on the original 2 part pilot yet the movie although shorter does contain extra footage unseen in the 2 hour pilot the pilot is much more of a pilot than the movie where as a pilot movie is normally the same 2 parter combined but the movie is actually a different edit with extras here and cuts there  worth a look even if you have the season 1 dvd set id still pick up a copy of the movie its still in some shops like virgin woolworths and the likes of mixed media stores although it generally needs ordering but it saves needing to buy online as many of us still dont do or trust online shopping but if you look around airwolfs in stores  airwolf was truly 1 of the 80s most under rated shows  a full size airwolf is currently being rebuilt for a helicopter museum  info and work in progress pictures are over at httpairwolforg also with airwolf mods for flashpoint and flight sim games it seams shes finally here to stay ',
 'does anyone happen to know where this fi

In [33]:
x_data[3:5]

array([[    37,     14,   2407, 201534,     96,  37314,    319,   7158,
        201534,   6469,   8828,   1085,     47,   9703,     20,    260,
            36,    455,      7,   7284,   1139,      3,  26494,   2633,
           203,    197,   3941,  12739,    646,      7,   7284,   1139,
             3,  11990,   7792,     46,  12608,    646,      7,   7284,
          1139,      3,   8593,     81,  36381,    109,      3, 201534,
          8735,    807,   2983,     34,    149,     37,    319,     14,
           191,  31906,      6,      7,    179,    109,  15402,     32,
            36,      5,      4,   2933,     12,    138,      6,      7,
           523,     59,     77,      3, 201534,     96,   4246,  30006,
           235,      3,    908,     14,   4702,   4571,     47,     36,
        201534,   6429,    691,     34,     47,     36,  35404,    900,
           192,     91,   4499,     14,     12,   6469,    189,     33,
          1784,   1318,   1726,      6, 201534,    410,     41, 

In [34]:
y_output[:5]

array([1, 1, 1, 1, 1])

#### Shuffle the data so the training instances are randomly fed to the RNN

In [35]:
np.random.seed(22)
shuffle_indices = np.random.permutation(np.arange(len(x_data)))

x_shuffled = x_data[shuffle_indices]
y_shuffled = y_output[shuffle_indices]

In [36]:
TRAIN_DATA = 5000
TOTAL_DATA = 6000

train_data = x_shuffled[:TRAIN_DATA]
train_target = y_shuffled[:TRAIN_DATA]

test_data = x_shuffled[TRAIN_DATA:TOTAL_DATA]
test_target = y_shuffled[TRAIN_DATA:TOTAL_DATA]

In [37]:
tf.reset_default_graph()

x = tf.placeholder(tf.int32, [None, MAX_SEQUENCE_LENGTH])
y = tf.placeholder(tf.int32, [None])

In [38]:
batch_size = 25
embedding_size = 50
max_label = 2

### Embeddings to represent words

These embeddings have been pre-built using GloVe a word vector embedding algorithm just like word2vec. The matrix will contain 400,000 word vectors, each with a dimensionality of 50.

* *saved_embeddings* This is a matrix which holds the embeddings for every word in the vocabulary. The values have been pre-loaded and were generated using the GloVe algorithm
* *embeddings* The embeddings for the words which are input as a part of one training batch

In [39]:
saved_embeddings = np.load('wordVectors.npy')
embeddings = tf.nn.embedding_lookup(saved_embeddings, x)

In [40]:
saved_embeddings

array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.013441 ,  0.23682  , -0.16899  , ..., -0.56657  ,  0.044691 ,
         0.30392  ],
       [ 0.15164  ,  0.30177  , -0.16763  , ..., -0.35652  ,  0.016413 ,
         0.10216  ],
       ...,
       [-0.51181  ,  0.058706 ,  1.0913   , ..., -0.25003  , -1.125    ,
         1.5863   ],
       [-0.75898  , -0.47426  ,  0.4737   , ...,  0.78954  , -0.014116 ,
         0.6448   ],
       [-0.79149  ,  0.86617  ,  0.11998  , ..., -0.29996  , -0.0063003,
         0.3954   ]], dtype=float32)

In [41]:
embeddings

<tf.Tensor 'embedding_lookup:0' shape=(?, 250, 50) dtype=float32>

In [42]:
lstmCell = tf.contrib.rnn.BasicLSTMCell(embedding_size)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)

### Results from an RNN of LSTM cells

(ouput, (**final_state**, other_state_info))

We're interested in the final state of this RNN because those are the encodings we feed into the prediction layer of our neural network

In [43]:
_, (encoding, _) = tf.nn.dynamic_rnn(lstmCell, embeddings, dtype=tf.float32)

In [44]:
encoding

<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 50) dtype=float32>

#### A densely connected prediction layer

* *activation=None* because the activation will be part of the tf.nn.sparse_softmax_cross_entropy_with_logits
* *cross_entropy* the loss function for probability distributions
* *max_label* the number of outputs of the prediction layer, here is 2, positive or negative

In [45]:
logits = tf.layers.dense(encoding, max_label, activation=None)

In [46]:
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(cross_entropy)

#### Find the output with the highest probability and compare against the true label

In [47]:
prediction = tf.equal(tf.argmax(logits, 1), tf.cast(y, tf.int64))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

In [48]:
optimizer = tf.train.AdamOptimizer(0.01)
train_step = optimizer.minimize(loss)

In [49]:
num_epochs = 20

In [50]:
init = tf.global_variables_initializer()

In [51]:
with tf.Session() as session:
    init.run()
    
    for epoch in range(num_epochs):
        
        num_batches = int(len(train_data) // batch_size) + 1
        
        for i in range(num_batches):
            # Select train data
            min_ix = i * batch_size
            max_ix = np.min([len(train_data), ((i+1) * batch_size)])

            x_train_batch = train_data[min_ix:max_ix]
            y_train_batch = train_target[min_ix:max_ix]
            
            train_dict = {x: x_train_batch, y: y_train_batch}
            
            
            session.run(train_step, feed_dict=train_dict)
            
            train_loss, train_acc = session.run([loss, accuracy], feed_dict=train_dict)

        test_dict = {x: test_data, y: test_target}
        test_loss, test_acc = session.run([loss, accuracy], feed_dict=test_dict)    
        print('Epoch: {}, Test Loss: {:.2}, Test Acc: {:.5}'.format(epoch + 1, test_loss, test_acc)) 

Epoch: 1, Test Loss: 0.69, Test Acc: 0.507
Epoch: 2, Test Loss: 0.69, Test Acc: 0.505
Epoch: 3, Test Loss: 0.69, Test Acc: 0.54
Epoch: 4, Test Loss: 0.57, Test Acc: 0.715
Epoch: 5, Test Loss: 0.7, Test Acc: 0.541
Epoch: 6, Test Loss: 0.55, Test Acc: 0.754
Epoch: 7, Test Loss: 0.56, Test Acc: 0.73
Epoch: 8, Test Loss: 0.54, Test Acc: 0.776
Epoch: 9, Test Loss: 0.57, Test Acc: 0.727
Epoch: 10, Test Loss: 0.57, Test Acc: 0.743
Epoch: 11, Test Loss: 0.6, Test Acc: 0.747
Epoch: 12, Test Loss: 0.63, Test Acc: 0.75
Epoch: 13, Test Loss: 0.65, Test Acc: 0.765
Epoch: 14, Test Loss: 0.69, Test Acc: 0.76
Epoch: 15, Test Loss: 0.72, Test Acc: 0.774
Epoch: 16, Test Loss: 0.75, Test Acc: 0.769
Epoch: 17, Test Loss: 0.81, Test Acc: 0.765
Epoch: 18, Test Loss: 0.82, Test Acc: 0.758
Epoch: 19, Test Loss: 0.86, Test Acc: 0.758
Epoch: 20, Test Loss: 0.97, Test Acc: 0.757


** D O N E ! **